In [144]:
# Import dependencies
import os
import subprocess
import json
from dotenv import load_dotenv
from web3 import Web3
from constants import *
from bit import *
from web3.middleware import geth_poa_middleware
from eth_account import Account


In [145]:
# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")
print(mnemonic)

team keen lock truck celery danger code squirrel engage mystery wrist announce


In [146]:
# Create a function called `derive_wallets`
def derive_wallets(mnemonic, coin, numderive):
    command = f"php ./derive -g --mnemonic=\"{mnemonic}\" --coin=\"{coin}\" --numderive=\"{numderive}\" --cols=index,path,privkey,pubkey,pubkeyhash,xprv,xpub,address --format=json"
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)
 

In [147]:
# Test the function derive_wallets
derive_wallets(mnemonic, ETH, 3)

[{'index': 0,
  'path': "m/44'/60'/0'/0/0",
  'privkey': '0xe4ac4343d43cf2523862a2ed1e6aa4ce37ba971081c8f60f4701c370ee9fbb2c',
  'pubkey': '02dc5ecce1df1c720656568f8a39341dbd3452c3bb03d2680cda4fd795d15de560',
  'pubkeyhash': '83150f74380af61b3ac11ec67f3e12963de82ecf',
  'xprv': 'xprvA2inXbstUjrFr5SQ3Tx9AAe4XcnN3dMT4Pr7h8ESEoS2DqMW3AeKcg7Fi2Uki5JKfupJ2xDKUmBtJiR41Re396EqfcCM1pgBVN943Z4p9om',
  'xpub': 'xpub6Fi8w7QnK7QZ4ZWs9VV9XJao5ecrT65JRcmiVWe3o8y16dgeahxaAURjZHSs5AQx6RPLtdzm43JrRPnDeYtgXr65NRHxh8xK2niWujnAxDa',
  'address': '0x2018B01C5a7ACa7ecf068345a8451D97eF90d4c2'},
 {'index': 1,
  'path': "m/44'/60'/0'/0/1",
  'privkey': '0xb7d21fb4b86fb69d76ceca4f653247e4065d96cf62c439ee6f40c15b106ab811',
  'pubkey': '023b209459ee3454f55a55c21646ece868e3b103396a4b7369a57b964b425dc9bc',
  'pubkeyhash': '47e87569a10e6379db3dde509c81367db9723150',
  'xprv': 'xprvA2inXbstUjrFtWdTALFTYAEu9PWeExbsN8vDHYAaDCnh2hFS7FguEjWMFmnT1irZ7j9wTpRjQDPg1MxKXHCq5vSZGiPQ7xPpPkoCjWH4Cnt',
  'xpub': 'xpub6Fi8w7QnK7QZ

In [148]:
# Create a dictionary object called coins to store the output from `derive_wallets`.

coins = {"btc-test": derive_wallets(mnemonic, BTCTEST, numderive), "eth": derive_wallets(mnemonic, ETH, numderive)}


In [149]:
# Test Final object:
btctest_privkey = coins[BTCTEST][0]['privkey']
eth_privkey = coins[ETH][0]['privkey']
print('BTC-Test private key:',json.dumps(btctest_privkey))
print('ETH private key:',json.dumps(eth_privkey))


BTC-Test private key: "cU5BsvDRcuwkg5rkPEDVbWYNzfxh9VX7ViKX9iwH9qGdDBdHZUJB"
ETH private key: "0xe4ac4343d43cf2523862a2ed1e6aa4ce37ba971081c8f60f4701c370ee9fbb2c"


In [150]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [151]:
# create a function to hold Ethereum 
eth_acc = priv_key_to_account(ETH, derive_wallets(mnemonic, ETH, 3)[0]['privkey'])
eth_acc

In [134]:
#Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, to, amount):
    if coin == ETH: 
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": to, "value": amount}
        )
        return {
            "from": account.address,
            "to": to,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
        }
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])


# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, to, amount):
    txn = create_tx(coin, account, to, amount)
    if coin == ETH:
        signed_txn = eth_acc.sign_transaction(txn)
        result = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
        print(result.hex())
        return result.hex()
    elif coin == BTCTEST:
        tx_btctest = create_tx(coin, account, to, amount)
        signed_txn = account.sign_transaction(txn)
        print(signed_txn)
        return NetworkAPI.broadcast_tx_testnet(signed_txn)

In [153]:
# create a function to hold Ethereum 
eth_acc = priv_key_to_account(ETH, derive_wallets(mnemonic, ETH, 3)[0]['privkey'])
eth_acc
eth_key = coins[ETH][0]['privkey']
eth_address = coins[ETH][1]['address']


In [155]:
send_tx(ETH, priv_key_to_account(ETH, eth_key),eth_address, 1)

0xcfe8601009ef1db1668c006fea892bf775a3705fab9f626109a4dc56b670d8c7


'0xcfe8601009ef1db1668c006fea892bf775a3705fab9f626109a4dc56b670d8c7'

In [156]:
eth_address

'0xF79E3d7EE2ec72A99A2Bd3a6b4cAFfa0Be185EB0'